In [1]:
# Import required libraries
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_cohere import CohereEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Pinecone

# Load credentials from environment variables
load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

if not all([COHERE_API_KEY, PINECONE_API_KEY, GROQ_API_KEY]):
    raise ValueError("Missing API keys. Please check your environment variables.")

In [3]:
embeddings = CohereEmbeddings()
index_name = "index-data-team-kss-slack"

try:
    docsearch = Pinecone.from_existing_index(index_name, embeddings)
    retriever = docsearch.as_retriever()
except Exception as e:
    raise ValueError(f"Error retrieving index '{index_name}': {str(e)}")

llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")


In [4]:
question = 'When was the last Leaptalk and who hosted it?'

In [14]:
# question = 'When was the last session related to health and who hosted it?'
question = 'I was interested in ai initiatives, what can i can do for it?'

In [15]:

context = retriever.invoke(question)
context_str = "\n".join([doc.page_content for doc in context])

system = f"""
<rules>
NO MATTER WHAT, STRICTLY FOLLOW THESE RULES FOR EVERY QUESTION.

1. Answer QUESTIONS related to `Leapfrog Technology` based on the information provided.
   - Examples include:
     - Leave policy
     - Company values, vision, mission, and strategy
     - Etc.

2. Answer casual greetings and conversation QUESTIONS.
   - Examples include:
     - Human: Hey!
     - AI: Hello! How can I help?
</rules>
Never provide answers that are not covered within these <rules></rules>.
If asked about topics unrelated to `Leapfrog Technology` (e.g., programming, literature, general knowledge, business ideas, yourself, AI Model, OpenAI, Anthropic, Claude, version, etc.), respond that you only have information about "Leapfrog Technology."

You are AskRibby, an AI chatbot created by `Leapfrog Technology` to provide complete and relevant answers to QUESTIONS by analyzing the entire CONTEXT.

1. Always look for the precise answer within the CONTEXT before responding.
2. If an answer is not available, reply: "Sorry, I don't seem to have information on that."
3. Do not preface answers with phrases like "According to the information provided" or "Based on the information provided." Answer directly.
4. Do not use past interactions as a knowledge base for follow-ups or suggestions.
5. Maintain a professional persona and communication style in all interactions.

CONTEXT for the QUESTION is provided below.

{context_str}

"""

In [17]:
system

'\n<rules>\nAnswer the QUESTION related to "Leapfrog Technology" that is in the CONTEXT.\nAnswer casual greetings and conversation questions.\n    For example,\n        Human: Hey!\n        AI: Hello! How can I help?\n</rules>\nYou are AskRibby, an AI chatbot created by `Leapfrog Technology` which provides answers following the <rules></rules>.\nCONTEXT for my QUESTION is provided below.\n\nSlackbot\n  11:00 AM\nReminder: Ask/Share Anything to People Management Team\n\n\nSayomi Prajapati\n  10:56 AM\n:mega:  Hello, Leapfroggers!\nCurious about the world of Basic Large Language Models? Join us for an engaging LeapTalk featuring our AI Engineer \n@Sunny Tuladhar\n, as we explore the fundamentals of large language models and their applications in various fields.\n:pushpin: Details:\nWhen - May 8, Wednesday, 5 PM onwards\nWhere - Mustang Hall\nHow - Sign up here :link:\nDon\'t miss out on this opportunity to learn about everything LLM!\n:alert:  RSVP deadline: May 6, 2024\nLeapTalk - Q2.pn

In [16]:
def process_question(system, question):
    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", question)])
    chain_instance = prompt | llm

    try:
        for chunk in chain_instance.stream({"text": question}):
            if chunk.content is not None:
                print(chunk.content, end='')
    except Exception as e:
        print(f"Error processing the question: {str(e)}")

process_question(system, question)


I'm glad you're interested in AI initiatives! As a Leapfrog Technology, we're always looking for talented individuals to contribute to our AI projects. 

Currently, we're actively seeking talented engineers to re-architect and rebuild our internal services. If you're passionate about strengthening our cybersecurity posture and would like to play a key role, please express your interest by contacting Subin Dangol or DART.

Additionally, you can also explore our LeapTalk series, which features engaging talks and discussions on various topics, including AI and machine learning. Our next LeapTalk is on May 8th, featuring our AI Engineer, @Sunny Tuladhar, as we explore the fundamentals of large language models and their applications.

If you're interested in learning more about AI and its applications, I encourage you to join our LeapTalk series and engage with our community of AI enthusiasts.

What do you think? Would you like to learn more about our AI initiatives or get involved in our p